In [97]:
import requests
import pandas as pd
from datetime import datetime, timedelta


In [103]:
def get_cuurent_weather(city, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'  # pour obtenir les données en degrés Celsius
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'description': data['weather'][0]['description'],
            'humidity': data['main']['humidity'],
            'pressure': data['main']['pressure'],
            'wind_speed': data['wind']['speed']
        }
        return weather
    else:
        return None

In [105]:
# Remplacez 'your_api_key' par votre propre clé API obtenue de OpenWeatherMap
api_key = '40ff5ede1101ddc6e4a3f0f819eded84'
city = 'Paris'
#base_url = 'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'
#response = requests.get(base_url, params=params)
weather = get_cuurent_weather(city, api_key)

if weather:
    print(f"Weather in {weather['city']}:")
    print(f"Temperature: {weather['temperature']}°C")
    print(f"Description: {weather['description']}")
    print(f"Humidity: {weather['humidity']}%")
    print(f"Pressure: {weather['pressure']} hPa")
    print(f"Wind Speed: {weather['wind_speed']} m/s")
else:
    print("Failed to get weather data", response.status_code)

Weather in Paris:
Temperature: 19.27°C
Description: broken clouds
Humidity: 70%
Pressure: 1014 hPa
Wind Speed: 7.2 m/s


## II. Prévisions météo à 5 jours


In [107]:
#This function sends a request to the weather forecast API for the specified city and returns a list of dictionaries with weather forecasts.
def previsions_meteo(city, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric', # pour obtenir les données en degrés Celsius
        'cnt': 40
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        print(data)
        weather = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'temp_min': data['temperature']['min'],
            'temp_max':data['temperature']['max'],
            'weather' : data['weather']['value'],
            'humidity': data['main']['humidity'],
            'pressure': data['main']['pressure'],
            #'wind_speed': data['wind']['speed'], 
            #'local_time': data['city']['timezone']
            'weather': data['weather'][0]['description'],
            'wind_speed': data['wind']['speed'],
            'local_time': datetime.utcfromtimestamp(data['dt']) + timedelta(seconds=data['city']['timezone'])
        }
        return weather
    else:
        return None

Les éléments à extraire comprennent des détails sur les conditions météorologiques à différents moments :

city : Nom de la ville pour laquelle les prévisions sont fournies.
temperature : Température actuelle en degrés Celsius.
temp_min : Température minimale prévue.
temp_max : Température maximale prévue.
pressure : Pression atmosphérique en hPa (hectopascals).
humidity : Humidité relative en pourcentage.
weather : Description textuelle des conditions météorologiques (par exemple, "ciel dégagé", "pluie légère").
wind_speed : Vitesse du vent en mètres par seconde.
local_time : Heure locale à laquelle la prévision s'applique, convertie à partir de l'heure UTC et ajustée au fuseau horaire de la ville.

In [109]:
def previsions_meteo(city, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/forecast"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric',  # pour obtenir les données en degrés Celsius
        'cnt': 40  # nombre maximal de prévisions à récupérer (5 jours à raison de 8 prévisions par jour)
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        #print(data)
        
        forecasts = []
        
        for dat in data['list']:
            forecast = {
                'city': data['city']['name'],
                'temperature': dat['main']['temp'],
                'temp_min': dat['main']['temp_min'],
                'temp_max': dat['main']['temp_max'],
                'pressure': dat['main']['pressure'],
                'humidity': dat['main']['humidity'],
                'weather': dat['weather'][0]['description'],
                '   wind_speed': dat['wind']['speed'],
                'local_time     ': datetime.utcfromtimestamp(dat['dt']) + timedelta(seconds=data['city']['timezone'])
            }
            forecasts.append(forecast)
        
        return forecasts
    else:
        return None

In [60]:
#This function iterates over a list of cities, calls previsions_meteo for each city, and compiles weather forecast data for all cities.
def get_forecast_data(cities, api_key):
    all_forecasts = []
    for city in cities:
        city_forecasts = previsions_meteo(city, api_key)
        if city_forecasts:
            all_forecasts.extend(city_forecasts)
        else:
            print(f"Failed to get weather data for {city}")
    return all_forecasts

def convert_to_dataframe(forecast_data):
    df = pd.DataFrame(forecast_data)
    return df

api_key = '40ff5ede1101ddc6e4a3f0f819eded84'
cities = ['London', 'New York', 'Paris']

forecast_data = get_forecast_data(cities, api_key)
df_forecast = convert_to_dataframe(forecast_data)

print(df_forecast.head())

     city  temperature  temp_min  temp_max  pressure  humidity     weather  \
0  London        18.54     18.54     18.94      1007        76  light rain   
1  London        18.25     18.21     18.25      1010        78  light rain   
2  London        19.84     19.84     19.84      1012        68  light rain   
3  London        17.18     17.18     17.18      1015        86  light rain   
4  London        15.20     15.20     15.20      1016        95  light rain   

      wind_speed     local_time       
0           4.77 2024-07-16 13:00:00  
1           4.42 2024-07-16 16:00:00  
2           3.45 2024-07-16 19:00:00  
3           2.31 2024-07-16 22:00:00  
4           1.93 2024-07-17 01:00:00  


In [111]:
df_forecast

,city,temperature,temp_min,temp_max,pressure,humidity,weather,wind_speed,local_time
0,London,18.54,18.54,18.94,1007,76,light rain,4.77,2024-07-16 13:00:00
1,London,18.25,18.21,18.25,1010,78,light rain,4.42,2024-07-16 16:00:00
2,London,19.84,19.84,19.84,1012,68,light rain,3.45,2024-07-16 19:00:00
3,London,17.18,17.18,17.18,1015,86,light rain,2.31,2024-07-16 22:00:00
4,London,15.20,15.20,15.20,1016,95,light rain,1.93,2024-07-17 01:00:00
...,...,...,...,...,...,...,...,...,...
115,Paris,22.59,22.59,22.59,1015,65,scattered clouds,3.38,2024-07-20 23:00:00
116,Paris,20.72,20.72,20.72,1016,59,broken clouds,2.03,2024-07-21 02:00:00
117,Paris,18.87,18.87,18.87,1016,61,overcast clouds,1.56,2024-07-21 05:00:00
118,Paris,19.07,19.07,19.07,1016,59,overcast clouds,2.27,2024-07-21 08:00:00


## III. Données de pollution de l'air actuelles

Vous devez récupérer les données actuelles de pollution de l'air pour la ville de Paris ainsi que pour deux autres villes de votre choix.

Les éléments à extraire comprennent des détails sur les niveaux de différents polluants à un moment donné :

city : Nom de la ville pour laquelle les données de pollution sont fournies.
aqi : Indice de qualité de l'air (Air Quality Index).
co : Concentration de monoxyde de carbone (CO) en µg/m³.
no : Concentration de monoxyde d'azote (NO) en µg/m³.
no2 : Concentration de dioxyde d'azote (NO2) en µg/m³.
o3 : Concentration d'ozone (O3) en µg/m³.
so2 : Concentration de dioxyde de soufre (SO2) en µg/m³.
pm2_5 : Concentration de particules fines (PM2.5) en µg/m³.
pm10 : Concentration de particules grossières (PM10) en µg/m³.
nh3 : Concentration d'ammoniac (NH3) en µg/m³.
local_time : Heure locale à laquelle les données de pollution s'appliquent, 

In [113]:
from datetime import datetime, timedelta
# Cette fonction parcourt la liste des villes, 
# appelle fetch_air_pollution_data pour chaque ville, 
# et compile les données de pollution de l'air pour toutes les villes.

def fetch_air_pollution_data(city, api_key):
    # First, we need to get the geographical coordinates of the city
    geocode_url = "http://api.openweathermap.org/geo/1.0/direct"
    geocode_params = {
        'q': city,
        'appid': api_key,
        'limit': 1
    }
    
    geocode_response = requests.get(geocode_url, params=geocode_params)
    if geocode_response.status_code == 200:
        geocode_data = geocode_response.json()
        if not geocode_data:
            return None
        lat = geocode_data[0]['lat']
        lon = geocode_data[0]['lon']
    else:
        return None

    # Now fetch the air pollution data using the coordinates
    air_pollution_url = "http://api.openweathermap.org/data/2.5/air_pollution"
    air_pollution_params = {
        'lat': lat,
        'lon': lon,
        'appid': api_key
    }

    air_pollution_response = requests.get(air_pollution_url, params=air_pollution_params)
    if air_pollution_response.status_code == 200:
        air_data = air_pollution_response.json()
        print(air_data)
        pollution = air_data['list'][0]
        timestamp = pollution['dt']
        local_time = datetime.utcfromtimestamp(timestamp) + timedelta(seconds=geocode_data[0].get('timezone', 0))

        air_pollution = {
            'city': city,
            'aqi': pollution['main']['aqi'],
            'co': pollution['components']['co'],
            'no': pollution['components']['no'],
            'no2': pollution['components']['no2'],
            'o3': pollution['components']['o3'],
            'so2': pollution['components']['so2'],
            'pm2_5': pollution['components']['pm2_5'],
            'pm10': pollution['components']['pm10'],
            'nh3': pollution['components']['nh3'],
            'local_time': local_time
        }
        return air_pollution
    else:
        return None
        

In [ ]:
# Cette fonction parcourt la liste des villes, 
# appelle fetch_air_pollution_data pour chaque ville, 
# et compile les données de pollution de l'air pour toutes les villes.
def get_pollution_data(cities, api_key):
    all_pollution_data = []
    for city in cities:
        city_pollution_data = fetch_air_pollution_data(city, api_key)
        if city_pollution_data:
            all_pollution_data.append(city_pollution_data)
        else:
            print(f"Failed to get air pollution data for {city}")
    return all_pollution_data

def convert_to_dataframe(pollution_data):
    df = pd.DataFrame(pollution_data)
    return df

api_key = '40ff5ede1101ddc6e4a3f0f819eded84'  
cities = ['Saint-Petersbourg', 'London', 'New York', 'Moscow', 'Berlin', 'Kolkata']

pollution_data = get_pollution_data(cities, api_key)
df_pollution = convert_to_dataframe(pollution_data)

print(df_pollution.head())

In [70]:
df_pollution

,city,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,local_time
0,Saint-Petersbourg,3,186.92,0.82,8.48,105.86,10.49,4.22,4.95,0.36,2024-07-15 14:54:04
1,London,2,161.89,1.01,6.26,90.12,4.95,4.04,4.33,0.53,2024-07-15 14:47:55
2,New York,5,367.17,1.63,19.54,183.11,11.56,19.92,23.21,1.54,2024-07-15 14:54:05
3,Moscow,3,263.69,3.86,38.04,113.01,17.64,5.18,6.62,0.55,2024-07-15 14:47:27
4,Berlin,3,163.56,0.31,2.29,125.89,1.86,3.81,4.12,1.50,2024-07-15 14:51:26


 ## IV. Prévisions de pollution de l'air à 4 jours


In [124]:

def fetch_air_pollution_forecast(city, api_key):
    # First, we need to get the geographical coordinates of the city
    geocode_url = "http://api.openweathermap.org/geo/1.0/direct"
    geocode_params = {
        'q': city,
        'appid': api_key,
        'limit': 1
    }
    
    geocode_response = requests.get(geocode_url, params=geocode_params)
    if geocode_response.status_code == 200:
        geocode_data = geocode_response.json()
        if not geocode_data:
            return None
        lat = geocode_data[0]['lat']   #geocode_response: response from API .
        lon = geocode_data[0]['lon']   #geocode_data: donnes en format JSON, including (lat)  (lon).
    else:
        return None

    # Now fetch the air pollution forecast data using the coordinates
    air_pollution_forecast_url = "http://api.openweathermap.org/data/2.5/air_pollution/forecast"
    air_pollution_forecast_params = {
        'lat': lat,
        'lon': lon,
        'appid': api_key
    }

    air_pollution_forecast_response = requests.get(air_pollution_forecast_url, params=air_pollution_forecast_params)# эта строка отправляет HTTP GET запрос к API для получения прогноза загрязнения воздуха, используя URL air_pollution_forecast_url и параметры air_pollution_forecast_params.
    if air_pollution_forecast_response.status_code == 200: # vérifie si la demande a abouti (le code de réponse 200 indique que la demande a abouti)
        forecast_data = air_pollution_forecast_response.json() 
        forecast_list = forecast_data['list'] 

        
        forecast_details = []#list dict
        for forecast in forecast_list:
            timestamp = forecast['dt']
            local_time = datetime.utcfromtimestamp(timestamp) + timedelta(seconds=geocode_data[0].get('timezone', 0))


# datetime.utcfromtimestamp(timestamp) : convertit le timestamp en un objet datetime en UTC (Coordinated Universal Time).
# geocode_data[0].get('timezone', 0) : suppose que les données de géocodage contiennent des informations sur le fuseau horaire de la ville. La méthode get récupère la valeur de la clé 'timezone' si elle existe, sinon elle renvoie 0.
# timedelta(seconds=geocode_data[0].get('timezone', 0)) : crée un objet timedelta représentant le décalage horaire en secondes.
# L'addition de ces objets donne l'heure locale correspondant au temps UTC, en tenant compte du fuseau horaire de la ville.
            
            air_pollution = {
                'city': city,
                'aqi': forecast['main']['aqi'],
                'co': forecast['components']['co'],
                'no': forecast['components']['no'],
                'no2': forecast['components']['no2'],
                'o3': forecast['components']['o3'],
                'so2': forecast['components']['so2'],
                'pm2_5': forecast['components']['pm2_5'],
                'pm10': forecast['components']['pm10'],
                'nh3': forecast['components']['nh3'],
                'local_time': local_time
            }
            forecast_details.append(air_pollution)
        return forecast_details
    else:
        return None

def get_forecast_data(cities, api_key):
    all_forecast_data = []
    for city in cities:
        city_forecast_data = fetch_air_pollution_forecast(city, api_key)
        if city_forecast_data:
            all_forecast_data.extend(city_forecast_data)
        else:
            print(f'Failed to get air pollution forecast data for {city}')
    return all_forecast_data        


api_key = '40ff5ede1101ddc6e4a3f0f819eded84'  
cities = ['Saint-Petersbourg', 'London', 'New York', 'Moscow', 'Berlin', 'Kazan']

forecast_data = get_forecast_data(cities, api_key)
df_forecast = convert_to_dataframe(forecast_data)

print(df_forecast.head())

                city  aqi      co    no   no2      o3    so2  pm2_5  pm10  \
0  Saint-Petersbourg    3  193.60  1.79  8.65  110.15  13.23   4.18  4.97   
1  Saint-Petersbourg    3  185.25  1.20  6.17  115.87   9.89   3.82  4.43   
2  Saint-Petersbourg    3  185.25  1.13  6.00  118.73   9.78   3.76  4.32   
3  Saint-Petersbourg    3  185.25  1.05  5.74  118.73   9.42   3.63  4.13   
4  Saint-Petersbourg    3  183.58  0.96  5.57  118.73   8.58   3.49  3.95   

    nh3          local_time  
0  0.26 2024-07-16 09:00:00  
1  0.35 2024-07-16 10:00:00  
2  0.40 2024-07-16 11:00:00  
3  0.44 2024-07-16 12:00:00  
4  0.47 2024-07-16 13:00:00  


In [89]:
df_forecast

,city,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,local_time
0,Saint-Petersbourg,3,186.92,0.82,8.48,105.86,10.49,4.22,4.95,0.36,2024-07-15 15:00:00
1,Saint-Petersbourg,3,191.93,1.05,9.85,101.57,11.68,4.54,5.27,0.43,2024-07-15 16:00:00
2,Saint-Petersbourg,2,195.27,0.71,11.65,92.98,12.40,4.59,5.32,0.46,2024-07-15 17:00:00
3,Saint-Petersbourg,2,196.93,0.26,13.37,85.12,12.99,4.59,5.36,0.44,2024-07-15 18:00:00
4,Saint-Petersbourg,2,201.94,0.01,15.08,75.82,13.95,4.64,5.48,0.40,2024-07-15 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...
571,Kazan,2,180.24,0.17,1.35,70.10,2.65,0.76,0.87,0.17,2024-07-19 10:00:00
572,Kazan,2,180.24,0.17,1.29,71.53,2.47,0.74,0.84,0.17,2024-07-19 11:00:00
573,Kazan,2,180.24,0.18,1.29,72.24,2.33,0.74,0.84,0.18,2024-07-19 12:00:00
574,Kazan,2,180.24,0.16,1.41,70.10,2.09,0.72,0.81,0.19,2024-07-19 13:00:00
